In [405]:
import network
import numpy as np
import copy
import imp
import mynet
imp.reload(network)
imp.reload(mynet)

<module 'mynet' from '/home/dan/Downloads/projects/neural_net/DeepLearningPython35/mynet.py'>

In [406]:
sizes = [4,2,3]
net = network.Network(sizes)

def printValues(listVals):
   for i in range(len(listVals)):
        print(listVals[i])
        print() 

In [407]:
sizes = [4,2,3]
net = network.Network(sizes)

print("original weights: (list of " + str(type(net.weights[0])) + ")")
print(type(net.weights[0]))
printValues(net.weights)
print("shape of net.weights[0]: " + str(net.weights[0].shape))


# manually set weights for connections between layers 0 and 1
net.weights[0]=np.array([[0.1, 0.2, 0.3, 0.4], [1.1,1.2,1.3,1.4]])
# same for layers 1 and 2
net.weights[1]=np.array([[0.5, -0.5], [0.6, -1], [0.7, -2]])

# manually set biases as well
net.biases[0] = np.array([[1.0], [1.0]])
net.biases[1] = np.array([[2.0], [2.0], [2.0]])

print("\nnew weights:")
printValues(net.weights)

print("new biases: (list of " + str(type(net.biases[0])) + ")")
print(type(net.biases[0]))
printValues(net.biases)

mine = mynet.Network(sizes)
mine.weights = copy.deepcopy(net.weights)
mine.biases = copy.deepcopy(net.biases)
mine.sizes = copy.deepcopy(net.sizes)

original weights: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[0.33021132 0.93187687 0.37859619 0.38582005]
 [0.23633977 0.64792759 0.08830015 0.25043338]]

[[0.88792344 0.39566582]
 [0.17283531 0.01274666]
 [0.90396271 0.13828046]]

shape of net.weights[0]: (2, 4)

new weights:
[[0.1 0.2 0.3 0.4]
 [1.1 1.2 1.3 1.4]]

[[ 0.5 -0.5]
 [ 0.6 -1. ]
 [ 0.7 -2. ]]

new biases: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[1.]
 [1.]]

[[2.]
 [2.]
 [2.]]



In [408]:
# calculate output for given input
# returns list of the activation at each layer
def evaluate(net, cur):
    #print("expected output = " + str(net.feedforward(cur))) # known solution

    # slow way of doing it:
#     for layer in range(net.num_layers-1):
#         tmp = []
#         for node in range(len(net.weights[layer])):
#             w = net.weights[layer][node]
#             b = net.biases[layer][node]
#             print("w = " + str(w))
#             print("b = " + str(b))
#             x = np.dot(w, cur) + b
#             x = network.sigmoid(x)
#             tmp.append([x[0]])
#         cur = tmp
#         print("new cur = " + str(cur) + "\n")

    activations = []
    #print("\n\nfaster way:")
    # fast way of doing it:
    a = cur # input
    activations.append(a)
    for b,w in zip(net.biases, net.weights):
        a = network.sigmoid(np.dot(w,a)+b)
        activations.append(a)
        #print(str(b) + "\n" + str(w) + "\n -> " + str(a) + "\n")
    return activations

In [409]:
# do backpropogation
# (currently just calculates errors at each node)
def backprop(cur, expected):
    activations = evaluate(net, cur)
    print("\nactivations:")
    print(activations)
    
    nabla_b = [np.zeros(b.shape, dtype=float) for b in net.biases]
    nabla_w = [np.zeros(w.shape, dtype=float) for w in net.weights]

    
    # array to store the calculated error at each node as we backpropogate
    # errors[l][j] = dC/dzl_0
    errors = []
    for n in net.sizes:
        errors.append(np.zeros((n, 1), dtype=float))
    #print('errors:')
    #print(errors)
    # traverse layers backwards
    # TODO: consider adding a dummy layer of to the beginning of the weights and biases vectors
    #       so the indicies make sense and match up with the errors array, etc
    for l in range(len(net.sizes)-1, 0, -1):
        index = l-1 # notes: subtract 1 from layer number to index into weights and biases arrays
        #print("at layer " + str(l))
        # traverse nodes in layer l
        for j in range(net.sizes[l]):
            al_j = activations[l][j]
            # if we're on the last layer
            if l == len(net.sizes)-1:
                #errors[l][j] = 2*(al_j-expected[j]) * al_j*(1-al_j) # BP1
                errors[l][j] = (al_j-expected[j]) * al_j*(1-al_j) # BP1
            else:
                # BP2
                total = 0
                for k in range(0, net.sizes[l+1]):
                    #print("k = " + str(k) + ", weight -> " + str(net.weights[index+1][k][j]))
                    #print("error -> " + str(errors[l+1][k][0]))
                    total += net.weights[index+1][k][j] * errors[l+1][k][0]
                errors[l][j] = al_j*(1-al_j) + total

            # TODO: as test update the bias and weights corresponding to the error just calculated
            # (not sure if doing this right)
            #net.biases[index][j] += -0.25 * errors[l][j]
            nabla_b[index][j] = errors[l][j]
            if (l != 0):
                for k in range(0, net.sizes[l-1]):
                    nabla_w[index][j] = errors[l][j]*activations[l-1][k]
                    #net.weights[index][j] += -0.25 * errors[l][j]*activations[l-1][k]
    
    return (nabla_b, nabla_w)
    #errors = np.zeros(tuple(net.sizes), dtype=float)
    #print("final errors:")
    #print(errors) # (TODO: hand check these)

In [410]:
# test backpropogation to see if we can get it to work for 2 or 3 training examples

data = np.array([[1], [2], [3], [4]]) # input
expected = np.array([[0.0], [1.0], [0.0]]) # known output for this input

# other training data
data2 = np.array([[-1], [-4], [-99], [0.3]])
expected2 = np.array([[0.333], [0.333], [0.333]])

data3 = np.array([[7.0], [4], [9], [6]])
expected3 = np.array([[0], [0], [1.0]])

nabla_b, nabla_w = net.backprop(data, expected)
print("nabla_b:")
print(nabla_b)
print("nabla_w:")
print(nabla_w)

print("----")
mine_b, mine_w = mine.backprop(data, expected)
print("nabla_b:")
print(mine_b)
print("nabla_w:")
print(mine_w)

# for i in range(4000):
#     if i%3 ==0:
#         backprop(data, expected)
#     elif i%3 ==1:
#         #backprop(data2, expected2)
#         pass
#     else:
#         backprop(data3, expected3)

their activations=
[array([[1],
       [2],
       [3],
       [4]]), array([[0.98201379],
       [0.99999917]]), array([[0.87984967],
       [0.8305048 ],
       [0.66539081]])]
nabla_b:
[array([[ 2.40024389e-03],
       [-2.65207413e-07]]), array([[ 0.09301263],
       [-0.02385926],
       [ 0.14814652]])]
nabla_w:
[array([[ 2.40024389e-03,  4.80048779e-03,  7.20073168e-03,
         9.60097557e-03],
       [-2.65207413e-07, -5.30414826e-07, -7.95622239e-07,
        -1.06082965e-06]]), array([[ 0.09133969,  0.09301255],
       [-0.02343012, -0.02385924],
       [ 0.14548193,  0.1481464 ]])]
----

activations:!!
[array([[1],
       [2],
       [3],
       [4]]), array([[0.98201379],
       [0.99999917]]), array([[0.87984967],
       [0.8305048 ],
       [0.66539081]])]
nabla_b:
[array([[ 2.40024389e-03],
       [-2.65207413e-07]]), array([[ 0.09301263],
       [-0.02385926],
       [ 0.14814652]])]
nabla_w:
[array([[ 2.40024389e-03,  4.80048779e-03,  7.20073168e-03,
         9.6009755